<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/pablo_camarillo_add_spark_setup/examples/notebooks/lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
[sudo] password for parallels: 
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .master("local[*]") \
  .appName("ITESO-2024-SparkIntroduction") \
  .config("spark.driver.bindAddress","localhost") \
  .config("spark.ui.port","4040") \
  .getOrCreate()

sc = spark.sparkContext

In [3]:
# Create an RDD with a list of sentences

sentences = [
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
    "Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.",
    "Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.",
    "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
    "Curabitur pretium tincidunt lacus. Nulla gravida orci a odio.",
    "Nullam varius, turpis et commodo pharetra, est eros bibendum elit, nec malesuada elit elit vel lectus.",
    "Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium.",
    "Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit.",
    "Sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.",
    "Neque porro quisquam est qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit.",
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."
]
sentences_rdd = sc.parallelize(sentences)

# Tokenize the sentences into words
words_rdd = sentences_rdd.flatMap(lambda line: line.split())

print(words_rdd.collect())

['Lorem', 'ipsum', 'dolor', 'sit', 'amet,', 'consectetur', 'adipiscing', 'elit.', 'Sed', 'do', 'eiusmod', 'tempor', 'incididunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliqua.', 'Ut', 'enim', 'ad', 'minim', 'veniam,', 'quis', 'nostrud', 'exercitation', 'ullamco', 'laboris', 'nisi', 'ut', 'aliquip', 'ex', 'ea', 'commodo', 'consequat.', 'Duis', 'aute', 'irure', 'dolor', 'in', 'reprehenderit', 'in', 'voluptate', 'velit', 'esse', 'cillum', 'dolore', 'eu', 'fugiat', 'nulla', 'pariatur.', 'Excepteur', 'sint', 'occaecat', 'cupidatat', 'non', 'proident,', 'sunt', 'in', 'culpa', 'qui', 'officia', 'deserunt', 'mollit', 'anim', 'id', 'est', 'laborum.', 'Curabitur', 'pretium', 'tincidunt', 'lacus.', 'Nulla', 'gravida', 'orci', 'a', 'odio.', 'Nullam', 'varius,', 'turpis', 'et', 'commodo', 'pharetra,', 'est', 'eros', 'bibendum', 'elit,', 'nec', 'malesuada', 'elit', 'elit', 'vel', 'lectus.', 'Sed', 'ut', 'perspiciatis', 'unde', 'omnis', 'iste', 'natus', 'error', 'sit', 'voluptatem', 'accusantium'

In [4]:
# Compute the frequency of each word
word_counts_rdd = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
print(word_counts_rdd.collect())

[('Lorem', 2), ('ipsum', 3), ('consectetur', 2), ('elit.', 2), ('Sed', 4), ('do', 2), ('labore', 2), ('magna', 2), ('quis', 2), ('exercitation', 2), ('ullamco', 2), ('laboris', 2), ('nisi', 2), ('aliquip', 2), ('ex', 2), ('commodo', 3), ('consequat.', 2), ('Duis', 1), ('aute', 1), ('in', 3), ('reprehenderit', 1), ('voluptate', 1), ('cillum', 1), ('fugiat', 1), ('nulla', 1), ('pariatur.', 1), ('sint', 1), ('sunt', 1), ('officia', 1), ('deserunt', 1), ('anim', 1), ('est', 3), ('laborum.', 1), ('lacus.', 1), ('varius,', 1), ('pharetra,', 1), ('bibendum', 1), ('malesuada', 1), ('unde', 1), ('omnis', 1), ('iste', 1), ('Nemo', 1), ('quia', 3), ('aspernatur', 1), ('fugit.', 1), ('dolores', 1), ('ratione', 1), ('porro', 1), ('dolor', 4), ('sit', 5), ('amet,', 3), ('adipiscing', 2), ('eiusmod', 2), ('tempor', 2), ('incididunt', 2), ('ut', 5), ('et', 3), ('dolore', 3), ('aliqua.', 2), ('Ut', 2), ('enim', 3), ('ad', 2), ('minim', 2), ('veniam,', 2), ('nostrud', 2), ('ea', 2), ('irure', 1), ('veli

In [5]:
# Find the most common word
most_common_word = word_counts_rdd.takeOrdered(1, key=lambda x: -x[1])
print("Most common word:", most_common_word)

Most common word: [('sit', 5)]


In [6]:
# Compute the average word length
total_length_rdd = words_rdd.map(lambda word: len(word)).reduce(lambda a, b: a + b)
total_words = words_rdd.count()
average_word_length = total_length_rdd / total_words if total_words > 0 else 0
print("Average word length:", average_word_length)

Average word length: 5.605555555555555


In [8]:
# Sort the RDD by word length in ascending order
sorted_words_by_length = words_rdd.sortBy(lambda word: len(word))

# Get the shortest word
shortest_word = sorted_words_by_length.first()

# Get the longest word
longest_word = sorted_words_by_length.sortBy(lambda word: len(word), ascending=False).first()

# Print the results
print("Shortest word:", shortest_word)
print("Longest word:", longest_word)


reprehenderit
Shortest word: a
Longest word: reprehenderit


In [9]:
#Alternative Optimized Solution Using reduce
# Find the shortest word
shortest_word = words_rdd.reduce(lambda a, b: a if len(a) < len(b) else b)

# Find the longest word
longest_word = words_rdd.reduce(lambda a, b: a if len(a) > len(b) else b)

print("Results using reduce")
print("Shortest word:", shortest_word)
print("Longest word:", longest_word)


Results using reduce
Shortest word: a
Longest word: reprehenderit


In [10]:
# Stop the SparkContext
sc.stop()